In [14]:
import cv2
import os
import numpy as np
import re

## 0. Funciones necesarias

In [15]:
def load_images_from_folder(folder):
    images = []
    number = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = modify_image(img)
            images.append(img)
            d = re.findall(r'[0-9]+', filename)
            number.append(int(d[0]))
    return images, number

In [16]:
def modify_image(image):

    #Procesamiento de imágenes
    gray = cv2.resize(image, (28,28))
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
    thresh_number = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 5)
    thresh_number = 255 - thresh_number

    #Normalización para CNN
    thresh_number = np.expand_dims(thresh_number, 2)
    thresh_number = thresh_number.astype('float32')
    thresh_number = thresh_number / 255.0
    
    return thresh_number

## 1. Lectura del directorio con las imágenes a predecir

In [18]:
images, number = load_images_from_folder('./images/numbers/')

#Dimensión para CNN
thresh_number = np.stack(images, axis = 0)

In [ ]:
cv2.imshow('number', thresh_number[0])
cv2.waitKey()

In [21]:
#Dimensión del conjunto de entrada a la CNN
thresh_number.shape

(27, 28, 28, 1)

## 2. Cargado del modelo entrenado

In [22]:
from keras.models import load_model

model = load_model('mnist_CNN_model.h5')

## 3. Predicción de la CNN

In [26]:
predictions = model.predict(x = thresh_number, steps = len(thresh_number), 
                            verbose = 0)

#Dimensión
predictions.shape

(27, 10)

## 4. Creación de formato de entrada del resolvedor

In [29]:
#Vector con 81 posiciones. Rellenar con predicciones
sudoku_matrix = np.zeros(81)

for i, j in zip(number, range(len(predictions))):
    sudoku_matrix[i] = predictions[j].argmax()
    
#Matriz 9x9
sudoku_matrix.resize(9,9)
sudoku_matrix

array([[0., 0., 4., 0., 2., 0., 8., 3., 0.],
       [0., 3., 0., 0., 0., 4., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [4., 0., 0., 0., 0., 6., 7., 0., 3.],
       [6., 0., 0., 7., 3., 8., 0., 0., 9.],
       [7., 0., 3., 5., 0., 0., 0., 0., 8.],
       [5., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 8., 0., 0., 0., 2., 0.],
       [0., 2., 1., 0., 9., 0., 6., 0., 0.]])

## 5. Guardado de la matriz

In [30]:
np.save('sudoku_matrix.npy', sudoku_matrix)